In [ ]:
#Import all the libraries we will use. 
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import scipy.optimize as optim
from glob import glob
import warnings
import astropy.time
warnings.filterwarnings('ignore')

In [ ]:
filenames = glob("VirgoA_22235/*.fits")
filenames.sort()
filenames = np.array(filenames)[:]
len(filenames)

In [ ]:
def noise_diode_calibration(hdu):
    noise_counts1 = hdu[2].data['Count1']
    noise_counts2 = hdu[2].data['Count2']
    noise_mjd = hdu[2].data['MJD']
    tcal1 = hdu[2].header['TCAL1']
    tcal2 = hdu[2].header['TCAL2']
    tcalsig1 = hdu[2].header['TCALSIG1']
    tcalsig2 = hdu[2].header['TCALSIG2']
    
    on1 = np.where(noise_counts1 > np.mean(noise_counts1))[0]
    off1 = np.where(noise_counts1 < np.mean(noise_counts1))[0]
    on_sig1 = np.std(noise_counts1[on1])
    off_sig1 = np.std(noise_counts1[off1])
    diff1 = np.mean(noise_counts1[on1])-np.mean(noise_counts1[off1])#Difference between the "on" and "off" in counts.
    diffsig1 = np.sqrt(on_sig1**2 + off_sig1**2)
    conv1 = diff1/tcal1
    convsig1 = conv1 * np.sqrt((diffsig1/diff1)**2 + (tcalsig1/tcal1)**2)
    
    on2 = np.where(noise_counts2 > np.mean(noise_counts2))[0]
    off2 = np.where(noise_counts2 < np.mean(noise_counts2))[0]
    on_sig2 = np.std(noise_counts1[on2])
    off_sig2 = np.std(noise_counts1[off2])
    diff2 = np.mean(noise_counts1[on2])-np.mean(noise_counts1[off2])#Difference between the "on" and "off" in counts.
    diffsig2 = np.sqrt(on_sig2**2 + off_sig2**2)
    conv2 = diff2/tcal2
    convsig2 = conv2 * np.sqrt((diffsig2/diff2)**2 + (tcalsig2/tcal2)**2)
    
    return conv1,convsig1,conv2,convsig2

def gaussian(x,A,centre,sigma):
    return A*np.exp(-(x-centre)**2/(2*sigma**2))

def baseline_correction(mjd,counts,limits):
    inds_on = np.where(np.logical_and(mjd > limits[0],mjd<limits[1]))[0]
    inds_off = np.where(np.logical_or(mjd < limits[0],mjd>limits[1]))[0]
    p = np.polyfit(mjd[inds_off],counts[inds_off],3)
    p1 = np.poly1d(p)
    return counts-p1(mjd) #Counts after polynomial subtraction.

def pointing_correction(north,centre,south): #Inputs - Amplitudes of gaussian fits.
    x=  [0,1,2]
    ydata = [north,centre,south]
    p0 = [max(ydata),x[np.argmax(ydata)],1]
    popt_pointing,pcov_pointing = optim.curve_fit(gaussian,xdata=x,ydata=[north,centre,south],p0=p0, maxfev=500000000)
    return popt_pointing[0]/centre
#date=[]

def plot_all(hdu,conv1,conv2,j,north_limits,centre_limits,south_limits):
    plt.figure(figsize=(16,4))
    name_arr = ['Noise diode','North scan','Centre scan','South Scan']
    for i in [1,2,3,4]:
        plt.subplot(1,4,i)
        mjd = (hdu[i+1].data['MJD']- min(hdu[i+1].data['MJD']))*24*3600 #Convert MJD to seconds.
        if i in [2,3,4]: #Drift scans.
            count_1 = hdu[i+1].data['Count1']/conv1#-np.mean(hdu[i+1].data['Count1']/conv1)
            count_2 = hdu[i+1].data['Count2']/conv2#-np.mean(hdu[i+1].data['Count2']/conv2)
        else: #Noise diode
            count_1 = hdu[i+1].data['Count1']/conv1-np.mean(hdu[i+1].data['Count1']/conv1)
            count_2 = hdu[i+1].data['Count2']/conv2-np.mean(hdu[i+1].data['Count2']/conv2)
        if i == 2:
            count1_corrected = baseline_correction(mjd,count_1,north_limits[j])
            count2_corrected = baseline_correction(mjd,count_2,north_limits[j])
            p0 = [max(count1_corrected),mjd[np.argmax(count1_corrected)],(max(mjd)-min(mjd))/4.] #[A guess, centre guess, sigma guess]
            popt_north1,pcov_north1 = optim.curve_fit(gaussian,xdata = mjd,ydata=count1_corrected,p0 = p0, maxfev=500000000)
            
            p0 = [max(count2_corrected),mjd[np.argmax(count2_corrected)],(max(mjd)-min(mjd))/4.] #[A guess, centre guess, sigma guess]
            popt_north2,pcov_north2 = optim.curve_fit(gaussian,xdata = mjd,ydata=count2_corrected,p0 = p0, maxfev=500000000)
            
            x = np.arange(min(mjd),max(mjd),0.01)
            
            plt.plot(x,gaussian(x,*popt_north1),color='red',label='count1_fit')
            plt.plot(x,gaussian(x,*popt_north2),color='blue',label='count2_fit')
            
            plt.scatter(mjd,count1_corrected,color = 'red',label = 'count1',s= 1) #s in scatter is the dot size.
            plt.scatter(mjd,count2_corrected,color = 'blue',label = 'count2',s=1)
        elif i == 3:
            count1_corrected = baseline_correction(mjd,count_1,centre_limits[j])
            count2_corrected = baseline_correction(mjd,count_2,centre_limits[j])
            
            p0 = [max(count1_corrected),mjd[np.argmax(count1_corrected)],(max(mjd)-min(mjd))/4.] #[A guess, centre guess, sigma guess]
            popt_centre1,pcov_centre1 = optim.curve_fit(gaussian,xdata = mjd,ydata=count1_corrected,p0 = p0, maxfev=500000000)
            
            p0 = [max(count2_corrected),mjd[np.argmax(count2_corrected)],(max(mjd)-min(mjd))/4.] #[A guess, centre guess, sigma guess]
            popt_centre2,pcov_centre2 = optim.curve_fit(gaussian,xdata = mjd,ydata=count2_corrected,p0 = p0, maxfev=500000000)
            
            x = np.arange(min(mjd),max(mjd),0.01)
            
            plt.plot(x,gaussian(x,*popt_centre1),color='red',label='count1_fit')
            plt.plot(x,gaussian(x,*popt_centre2),color='blue',label='count2_fit')
            
            
            plt.scatter(mjd,count1_corrected,color = 'red',label = 'count1',s=1)
            plt.scatter(mjd,count2_corrected,color = 'blue',label = 'count2',s=1)
        elif i == 4:
            count1_corrected = baseline_correction(mjd,count_1,south_limits[j])
            count2_corrected = baseline_correction(mjd,count_2,south_limits[j])
            
            p0 = [max(count1_corrected),mjd[np.argmax(count1_corrected)],(max(mjd)-min(mjd))/4.] #[A guess, centre guess, sigma guess]
            popt_south1,pcov_south1 = optim.curve_fit(gaussian,xdata = mjd,ydata=count1_corrected,p0 = p0, maxfev=500000000)
            
            p0 = [max(count2_corrected),mjd[np.argmax(count2_corrected)],(max(mjd)-min(mjd))/4.] #[A guess, centre guess, sigma guess]
            popt_south2,pcov_south2 = optim.curve_fit(gaussian,xdata = mjd,ydata=count2_corrected,p0 = p0, maxfev=500000000)
            
            x = np.arange(min(mjd),max(mjd),0.01)
            
            plt.plot(x,gaussian(x,*popt_south1),color='red',label='count1_fit')
            plt.plot(x,gaussian(x,*popt_south2),color='blue',label='count2_fit')
            
            
            plt.scatter(mjd,count1_corrected,color = 'red',label = 'count1',s=1)
            plt.scatter(mjd,count2_corrected,color = 'blue',label = 'count2',s=1)
        else:
            plt.plot(mjd,count_1,color = 'red',label = 'count1')
            plt.plot(mjd,count_2,color = 'blue',label = 'count2')
        plt.title(name_arr[i-1])
        if i == 2:
            plt.vlines(x = north_limits[j][0],ymin = min(count1_corrected),ymax = max(count1_corrected),color='black')
            plt.vlines(x = north_limits[j][1],ymin = min(count1_corrected),ymax = max(count1_corrected),color='black')
        plt.legend()
    try:
        
        pointing1 = pointing_correction(popt_north1[0],popt_centre1[0],popt_south1[0])
        pointing2 = pointing_correction(popt_north2[0],popt_centre2[0],popt_south2[0])
        
        #if pointing1>0.9 and pointing1<1.4 and pointing2>0.9 and pointing2<1.4:
        #date.append(hdu[0].header['DATE'])
        print('The Date of creation of this file was {}' .format(hdu[0].header['DATE']))
        print('The pointing correction for this scan: Pol1 %.3f -- Pol2 %.3f'%(pointing1,pointing2))
        print('The peak antenna temperature is: Pol1 %.3f K-- Pol2 %.3f K'
          %(popt_centre1[0]*pointing1,popt_centre2[0]*pointing2))
        plt.suptitle('Plot index - %d'%(j))
        plt.show()
        plt.close()
        return [popt_centre1[0]*pointing1,np.sqrt(np.diag(pcov_centre1))[0]/popt_centre1[0],popt_centre2[0]*pointing2,np.sqrt(np.diag(pcov_centre2))[0]/popt_centre2[0]]
    except:
        
        print('Pointing correction could not be calculated. Please check the data.')
        plt.suptitle('Plot index - %d'%(j))
        plt.show()
        plt.close()
    return [[],[],[],[]]
    
def ott(nu,a,b,c):
    return 10**(a+b*np.log10(nu)+c*np.log10(nu)**2)
    
#Noise diode calibration.
    #Input - hdu
    #What does the function do? - calculate the counts/K conversion for both polarizations.
    #Output - the counts/K conversion.
    


In [ ]:
good_plot_ind =[1, 2, 3, 4,  6, 7, 8, 10, 11,  13,  15, 16, 26, 34, 35, 37, 38, 41, 42, 50, 53, 64, 65, 67, 68, 71, 72, 74, 75, 76, 77, 78, 78, 79, 80, 84, 85, 86, 89, 90, 91, 94, 95, 97, 98, 99, 100, 103, 110, 118, 133, 134, 138, 141, 143, 146, 154, 160, 161, 163, 164, 166, 169, 170, 173, 174, 177, 179, 180, 181, 182, 183,  186, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 212, 213, 214, 216, 217, 218, 221, 222, 223, 225, 226, 227, 228, 229, 230, 231, 233, 235, 236, 238, 239, 240, 241, 243, 245, 246, 247, 249, 250, 252, 253, 254, 256, 257, 264, 267, 268, 269, 270, 271, 272, 274, 275, 277, 279, 282, 284, 285, 286, 288, 289, 290, 291, 292, 293, 295, 296, 301, 302, 304, 305, 307, 308, 331, 340, 343, 350, 352, 358, 369, 396, 375, 377, 396, 407, 409, 411, 416, 417, 420, 496, 497, 498, 499, 501, 502, 504, 505, 506, 507, 510, 511, 512, 513, 514, 516, 517, 518, 523, 524, 526, 527, 528, 529, 532, 533, 534, 535, 536, 544, 546, 547, 558, 563, 565, 571, 572, 582, 586, 592, 601, 603, 607, 611, 612, 613, 614, 616, 617, 618, 621, 622, 625, 627, 628, 629, 630, 631, 632, 633, 635, 639, 640, 642, 643, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 657, 658, 659, 660, 661, 662, 663, 664, 672, 673, 674, 677, 678, 680, 682, 683, 685, 686, 687, 688, 689, 691, 692, 693, 694, 695, 698, 699, 706, 707, 708, 710, 714, 717, 718, 720, 725, 735, 737, 739, 740, 745, 746, 747, 748, 749, 751, 752, 753, 754, 755, 756, 757, 758, 762, 801, 807, 810, 816, 819, 822, 823, 824, 826, 839, 843, 846, 847, 848, 850, 856, 860, 861, 862, 863, 865, 869, 877, 886, 909, 910, 912, 913, 916, 920, 921, 923, 928, 939, 941, 942, 946, 947, 948, 949, 953, 955, 957, 958, 963, 964, 965, 966, 967, 968, 970, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 986, 988, 989, 990, 991, 992, 993, 994, 996, 997, 998, 999, 1000, 1001, 1003, 1008, 1009, 1010, 1011, 1012, 1015, 1018, 1019, 1024, 1026, 1029, 1030, 1031, 1032, 1035, 1036, 1037, 1038, 1041, 1042, 1043, 1044, 1045, 1047, 1053, 1055, 1057, 1059, 1060, 1062, 1066, 1067, 1068, 1069, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1083, 1085, 1088, 1089, 1090, 1091, 1092, 1097, 1098, 1101, 1102, 1106, 1118, 1119, 1122, 1130, 1133, 1134, 1135, 1137, 1138, 1139, 1143, 1151, 1152, 1153, 1157, 1158, 1159, 1161, 1173, 1174, 1176, 1177, 1181, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1221, 1224, 1225, 1226, 1227, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1271, 1273, 1280, 1285, 1286, 1289, 1293, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1305, 1308, 1309, 1310, 1314, 1316, 1318, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1332, 1338, 1339, 1344, 1346, 1347, 1350, 1352, 1358, 1359, 1361, 1363, 1365, 1366, 1367, 1368, 1369, 1370, 1372, 1377, 1381, 1383, 1392, 1393, 1394, 1395, 1396, 1398, 1399, 1402, 1403, 1406, 1051, 1418, 1419, 1424, 1427, 1428, 1429, 1430, 1431, 1432, 1436, 1438, 1439, 1440, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1454, 1455, 1456, 1457, 1459, 1460, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470, 1471, 1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490, 1496, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1506, 1510, 1511, 1512, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1534, 1536, 1539, 1546, 1548, 1549, 1550, 1554, 1557, 1561, 1562, 1563, 1564, 1568, 1569, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1585, 1586, 1587, 1588, 1597, 1598, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1612, 1613, 1615, 1617, 1618, 1623, 1625, 1627, 1628, 1633, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1651, 1652, 1653, 1654, 1655, 1647, 1658, 1659, 1660, 1661, 1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1679, 1681, 1682, 1691, 1692, 1693, 1695, 1696, 1697, 1698, 1700, 1701, 1702, 1706, 1709, 1712, 1713, 1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 1732, 1734, 1736, 1737, 1738, 1739, 1740, 1742, 1746, 1747, 1749, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1766, 1767, 1769, 1770, 1774, 1775, 1776, 1778, 1779, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1790, 1791, 1793, 1794, 1795, 1796, 1797, 1798, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1812, 1813, 1814, 1815, 1816, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1853, 1854, 1855, 1856, 1857, 1858, 1862, 1863, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1901, 1902, 1903, 1905, 1906, 1907, 1908, 1911, 1913, 1914, 1916, 1917, 1921, 1923, 1929, 1931, 1932, 1935, 1936, 1937, 1938, 1939, 1944, 1945, 1948, 1949, 1950, 1951, 1954, 1955, 1956, 1958, 1959, 1960, 1962, 1963, 1964, 1965, 1968, 1969, 1971, 1972, 1973, 1974, 1975, 1976, 1979, 1980, 1981, 1982, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1992, 1993, 1994, 1995, 1996, 1999, 2007, 2009, 2013, 2014, 2015, 2020, 2021, 2022, 2027, 2028, 2029, 2030, 2036, 2038, 2039, 2040, 2041, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2057, 2058, 2060, 2061, 2062, 2063, 2064, 2067, 2068, 2070, 2072, 2073, 2076, 2078, 2079, 2080, 2083, 2084, 2085, 2086, 2088, 2090, 2091, 2092, 2093, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2130, 2131, 2132, 2133, 2134, 2135, 2138, 2141, 2147, 2148, 2149, 2150, 2151, 2152, 2156, 2157, 2159, 2161, 2163, 2171, 2174, 2175, 2176, 2177, 2179, 2182, 2184, 2186, 2187, 2188, 2189, 2199, 2202, 2206, 2210, 2230, 2239, 2240, 2246, 2264, 2286, 2292, 2795, 2299, 2303, 2308, 2310, 2311, 2312, 2315, 2317, 2318, 2320, 2327, 2331, 2332, 2335, 2339, 2340, 2345, 2351, 2353, 2358, 2360, 2362, 2363, 2364, 2366, 2369, 2371, 2374, 2375, 2376, 2377, 2378, 2380, 2381, 2382, 2384, 2388, 2389, 2390, 2391, 2395, 2396, 2397, 2399, 2400, 2401, 2402, 2411, 2412, 2430, 2433, 2436, 2437, 2438, 2439, 2440, 2444, 2452, 2460, 2462, 2464, 2503, 2507, 2509, 2510, 2515, 2517, 2520, 2521, 2524, 2525, 2528, 2536, 2538, 2542, 2543, 2545, 2546, 2550, 2552, 2556, 2557, 2558, 2562, 2567, 2568, 2569]#, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1271, 1299, 1300, 1306, 1322, 1323, 1329, 1339, 1358, 1359, 1363, 1365, 1368, 1369, 1371, 1372, 1381, 1382, 1383, 1392, 1393, 1394, 1395, 1397, 1398, 1404, 1406, 1407, 1418, 1419, 1424, 1428, 1429, 1430, 1431, 1432, 1436, 1438, 1439, 1443, 1444, 1445, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470, 1471, 1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1506, 1510, 1511, 1512, 1521, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1536, 1548, 1549, 1550, 1557, 1559, 1561, 1562, 1563, 1568, 1569, 1570, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1585, 1586, 1587, 1588, 1599, 1601, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1612, 1615, 1616, 1617, 1618, 1619, 1620, 1623, 1627, 1628, 1633, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1647, 1649, 1651, 1652, 1653, 1654, 1657, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1665, 1666, 1667, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1679, 1681, 1682, 1684, 1696, 1697, 1698, 1710, 1712, 1719, 1724, 1726, 1727, 1735, 1736, 1739, 1740, 1746, 1747, 1749, 1751, 1753, 1754, 1755, 1757, 1758, 1761, 1766, 1769, 1776, 1778, 1779, 1782, 1785, 1790, 1793, 1796, 1797, 1798, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1812, 1813, 1814, 1815, 1816, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1844, 1845, 1848, 1849, 1850, 1851, 1853, 1854, 1855, 1856, 1857, 1858, 1862, 1863, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1901, 1902, 1905, 1906, 1911, 1938, 1939, 1945, 1949, 1955, 1959, 1973, 1974, 1975, 1976, 1981, 1986, 1987, 1989, 1992, 1993, 1994, 2000, 2001, 2004, 2009, 2013, 2014, 2015, 2020, 2021, 2022, 2027, 2028, 2029, 2030, 2034, 2036, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2058, 2060, 2061, 2062, 2063, 2064, 2068, 2070, 2072, 2073, 2079, 2080, 2083, 2084, 2085, 2086, 2088, 2090, 2091, 2092, 2093, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2130, 2131, 2132, 2133, 2134, 2135, 2141, 2149, 2150, 2151, 2152, 2157, 2165, 2175, 2189, 2202, 2371, 2372, 2375, 2376, 2377, 2381, 2382, 2384, 2395, 2456, 2542, 2543, 2572]

In [ ]:
good2= [ 1690, 1691, 1692, 1693, 1695, 1696, 1697, 1701, 1704, 1707, 1708, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1727, 1729, 1731, 1732, 1733, 1734, 1735, 1737, 1741, 1742, 1744, 1746, 1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1761, 1762, 1764, 1765, 1769, 1770, 1771, 1773, 1774, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1785, 1786, 1788, 1789, 1790, 1791, 1792, 1793, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1807, 1808, 1809, 1810, 1811, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1840, 1841, 1842, 1843, 1844, 1845, 1846, 1848, 1849, 1850, 1851, 1852, 1853, 1857, 1858, 1860, 1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1893, 1894, 1896, 1897, 1898, 1900, 1901, 1902, 1903, 1906, 1908, 1909, 1911, 1912, 1916, 1918, 1924, 1926, 1927, 1930, 1931, 1932, 1933, 1934, 1939, 1940, 1943, 1944, 1945, 1946, 1949, 1950, 1951, 1953, 1954, 1955, 1957, 1958, 1959, 1960, 1963, 1964, 1966, 1967, 1968, 1969, 1970, 1971, 1974, 1975, 1976, 1977, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1987, 1988, 1989, 1990, 1991, 1994, 2002, 2004, 2008, 2009, 2010, 2015, 2016, 2017, 2022, 2023, 2024, 2025, 2031, 2033, 2034, 2035, 2036, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2052, 2053, 2055, 2056, 2057, 2058, 2059, 2062, 2063, 2065, 2067, 2068, 2071, 2073, 2074, 2075, 2078, 2079, 2080, 2081, 2083, 2085, 2086, 2087, 2088, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2125, 2126, 2127, 2128, 2129, 2130, 2133, 2136, 2142, 2143, 2144, 2145, 2146, 2147, 2151, 2152, 2154, 2156, 2158, 2166, 2169, 2170, 2171, 2172, 2174, 2177, 2179, 2181, 2182, 2183, 2184, 2194, 2197, 2201, 2205, 2225, 2234, 2235, 2241, 2259, 2281, 2287, 2790, 2294, 2298, 2303, 2305, 2306, 2307, 2310, 2312, 2313, 2315, 2322, 2326, 2327, 2330, 2334, 2335, 2340, 2346, 2348, 2353, 2355, 2357, 2358, 2359, 2361, 2364, 2366, 2369, 2370, 2371, 2372, 2373, 2375, 2376, 2377, 2379, 2383, 2384, 2385, 2386, 2390, 2391, 2392, 2394, 2395, 2396, 2397, 2406, 2407, 2425, 2428, 2431, 2432, 2433, 2434, 2435, 2439, 2447, 2455, 2457, 2459, 2498, 2502, 2504, 2505, 2510, 2512, 2515, 2516, 2519, 2520, 2523, 2531, 2533, 2537, 2538, 2540, 2541, 2545, 2547, 2551, 2552, 2553, 2557, 2562, 2563, 2564]
print([i+5 for i in good2])

In [ ]:
print([i for i in range(2371,2374)])

In [ ]:
print([i for i in range(1287,12)])

In [ ]:
good_plot_ind.pop(585)

In [ ]:
len(good_plot_ind)

In [ ]:
%matplotlib inline

#Good plots:
#good_plot_ind = [2,3,4,5,6,7,8,9,11,12,13,14,16,17] #it might be better to change this this to bad plot ind.
north_limits = [[7,30]]*3+[[5,30]]*2572
centre_limits = [[5,30]]*2572
south_limits = [[5,30]]*2572
#J is going from 0 - 19.
#But we only want to use it when it is in good_plot_ind
ott_a =4.484
ott_b = -0.603
ott_c = -0.0280

PSS1 = []
PSS2 = []
PSS1_frac = []
PSS2_frac = []
mjd_estimate = []
date = []

counter = 0
for j,filename in enumerate(filenames):
    if j in good_plot_ind: #it might be better to use if j not in bad_plot_ind:
        #try:
        hdu = fits.open(filename)
    #         hdu = hdu.replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna()
        datetime,hourmin = filename.split('/')[-1].split('_')[0:2]
        year,dayofyear = datetime.split('d')
        mjd_est = float(year)*float(dayofyear)
        freq = hdu[2].header['CENTFREQ']
        flux_density = ott(freq,ott_a,ott_b,ott_c)
        conv1,convsig1,conv2,convsig2 = noise_diode_calibration(hdu)

        #print("Counts/K conversion factors - Pol1 - %d counts/K, Pol2 - %d counts/K"%(conv1,conv2))
        print("Counts/K conversion factors - Pol1 - {:.6} ± {:.6} counts/K, Pol2 - {:.6} ± {:.6} counts/K".format(conv1,convsig1,conv2,convsig2))


        peak1,frac_peak1,peak2,frac_peak2 = plot_all(hdu,conv1,conv2,counter,north_limits,centre_limits,south_limits)

    #except ValueError:
        #continue

    #if peak2:
        if peak1:
            noise_diode_frac1 = convsig1/conv1
            noise_diode_frac2 = convsig2/conv2

            frac1 = np.sqrt(noise_diode_frac1**2 + frac_peak1**2)
            frac2 = np.sqrt(noise_diode_frac2**2 + frac_peak2**2)
            date.append(hdu[0].header['DATE'])
            mjd_estimate.append(mjd_est)
            PSS1.append(flux_density/peak1) #These are our main results. 
            PSS1_frac.append(frac1)
            PSS2.append(flux_density/peak2)
            PSS2_frac.append(frac2)
        counter+=1


In [ ]:
plt.figure(figsize=(7,5))
# plt.scatter(mjd_estimate,PSS1,label='LCP')
# plt.scatter(mjd_estimate,PSS2,label='RCP')

plt.errorbar(mjd_estimate,PSS1,yerr=np.array(PSS1)*np.array(PSS1_frac),label='LCP',linestyle='None',marker="o")
plt.errorbar(mjd_estimate,PSS2,yerr=np.array(PSS2)*np.array(PSS2_frac),label='LCP',linestyle='None',marker="o")

plt.xlabel('MJD Estimate')
plt.ylabel('PSS (Jy/K)')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
#plt.ylim(0,200)
plt.legend()
plt.savefig("pss_gooderr3_plots.png")

In [ ]:
plt.subplot(121)
plt.figure(figsize=(7,5))
plt.scatter(mjd_estimate,PSS1,label='LCP', marker='o')
plt.scatter(mjd_estimate,PSS2,label='RCP', marker='o')

# plt.errorbar(mjd_estimate,PSS1,yerr=np.array(PSS1)*np.array(PSS1_frac),label='LCP',linestyle='None',marker="o")
# plt.errorbar(mjd_estimate,PSS2,yerr=np.array(PSS2)*np.array(PSS2_frac),label='LCP',linestyle='None',marker="o")

plt.xlabel('MJD Estimate')
plt.ylabel('PSS (Jy/K)')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend()

plt.subplot(122)

plt.errorbar(mjd_estimate,PSS1,yerr=np.array(PSS1)*np.array(PSS1_frac),label='LCP',linestyle='None',marker="o")
plt.errorbar(mjd_estimate,PSS2,yerr=np.array(PSS2)*np.array(PSS2_frac),label='LCP',linestyle='None',marker="o")

plt.xlabel('MJD Estimate')
plt.ylabel('PSS (Jy/K)')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
#plt.ylim(0,200)
plt.legend()

#plt.savefig("pss_gooderr3_plots.png")
#plt.savefig("pss_good3_plots.png")

In [ ]:
plt.figure(figsize=(16,5))
plt.subplot(121)


plt.scatter(mjd_estimate,PSS1,label='LCP', marker='o')
plt.scatter(mjd_estimate,PSS2,label='RCP', marker='o')


plt.title("PSS_Values",fontsize=14)
plt.xlabel('MJD',fontsize=14)
plt.ylabel('PSS',fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
#plt.show()
#plt.close()

######################### CHANNEL-2 #######################
plt.subplot(122)

plt.errorbar(mjd_estimate,PSS1,yerr=np.array(PSS1)*np.array(PSS1_frac),label='LCP',linestyle='None',marker="o")
plt.errorbar(mjd_estimate,PSS2,yerr=np.array(PSS2)*np.array(PSS2_frac),label='LCP',linestyle='None',marker="o")

plt.title("PSS_Values + Error",fontsize=14)
plt.xlabel('MJD',fontsize=14)
plt.ylabel('PSS',fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
#plt.legend()
#plt.show()
plt.savefig("pss.png")

In [ ]:
print(len(PSS1))
print(len(PSS2))
print(len(PSS1_frac))
print(len(PSS2_frac))

In [ ]:
import pandas as pd

In [ ]:
s="&"
k="\\\\"
#for i in PSS1:
for j in range(len(PSS1)):

    data = pd.DataFrame({"Date" :date, "join0" :s, "PSS1" :PSS1,"join1" :s, 
                         "UNC1" :np.array(PSS1)*np.array(PSS1_frac), "join2" :s, 
                         "PSS2" :PSS2, "join3" :s, "UNC2" :np.array(PSS2)*np.array(PSS2_frac),"line" :k,})
#data
#pd.options.display.float_format = "{:.2f}".format  
#print(data.to_string(index=False))
#data.to_csv("data1.csv")

In [ ]:
pd.read_csv("data1.csv")

In [ ]:
# # Now we have to inspect the data, as some data can be bed
# # and we can not work with them, for that lets
#     # Plot the temp calibration
#     # and the three drift scans
# %matplotlib inline    
# def plot_all(hdu):
#     plt.figure(figsize=(16,4))
#     name_arr = ['Noise diode', 'North Scan', 'Centre Scan', 'South Scan']
#     for i in [1,2,3,4]:
#         plt.subplot(1,4,i)
#         mjd = (hdu[i+1].data['MJD'] - min(hdu[i+1].data['MJD']))*24*3600 # From days to seconds
#         count_1 = hdu[i+1].data['Count1'] - np.mean(hdu[i+1].data['Count1'])
#         count_2 = hdu[i+1].data['Count2'] - np.mean(hdu[i+1].data['Count2'])
#         plt.plot(mjd,count_1, color = 'red', label = 'Count1')
#         plt.plot(mjd,count_2, color = 'blue', label = 'Count2')
#         plt.title(name_arr[i-1])
#         plt.legend()
#     plt.suptitle('Plot index - %d'%(j))
   
#     plt.show()
#     plt.close()


In [ ]:
s="&"
k="\\\\"
#for i in PSS1:
for j in range(len(PSS1)):

    data = pd.DataFrame({"Date" :date,  "PSS1" :PSS1, 
                         "PSS1_UNCERTAINTY" :np.array(PSS1)*np.array(PSS1_frac),  
                         "PSS2" :PSS2,  "PSS2_UNCERTAINTY" :np.array(PSS2)*np.array(PSS2_frac)})
data
pd.options.display.float_format = "{:.2f}".format  
print(data.to_string(index=True))
data.to_csv("PSS_VALUES_VIRGO_A_22235.csv")